In [1]:
import sys
sys.path.append("..")

In [2]:
import os
from models.frequency_model import FrequencyModel
import tensorflow as tf
import numpy as np

# Define parametes

In [3]:
train_parameters = {
    "lr": 0.0001,
    "decay_steps": 300,
    "decay_rate": 0.9,
    "dropout_prob": 0.2
}

model_parameters = {
    "phonemes_embedding_size": 16,
    "speaker_embedding_size": 16,
    "num_bidirectional_layers": 2,
    "num_bidirectional_units": 16,
    "conv_widths": [2, 2],
    "output_dimension": 2    
}

input_vocab_size = 50
num_speakers = 10

num_steps = 2

save_energy = 2

# Train model

In [4]:
with tf.Session() as sess:
    phonemes = tf.placeholder(tf.int32, [None, 200])
    phonemes_seq_len = tf.placeholder(tf.int32, [None])
    speaker_ids = tf.placeholder(tf.int32, [None])
    voiced_target = tf.placeholder(tf.int32, [None, 200])
    frequency_target = tf.placeholder(tf.float32, [None, 200])
    
    prediction_phonemes = tf.placeholder(tf.int32, [None, 200])
    prediction_phonemes_seq_len = tf.placeholder(tf.int32, [None])
    prediction_speaker_ids = tf.placeholder(tf.int32, [None])

    model = FrequencyModel(
        input_vocab_size, num_speakers,
        model_parameters
    )
    
    train_op_tf, loss_tf, global_step_tf, summary_tf = model.build_train_operations(
        phonemes, phonemes_seq_len, speaker_ids, voiced_target, frequency_target, train_parameters
    )
    
    prediction_voiced, prediction_frequencies = model.build_prediction(
        prediction_phonemes, prediction_phonemes_seq_len, prediction_speaker_ids, True
    )

    train_writer = tf.summary.FileWriter('../log/train_grapheme_to_phoneme_model_notebook/train', sess.graph)
    
    tf.global_variables_initializer().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    saver = tf.train.Saver(max_to_keep=3, keep_checkpoint_every_n_hours=3)
    
    for _ in xrange(num_steps):
        
        out = sess.run([
            train_op_tf,
            loss_tf,
            global_step_tf,
            summary_tf,
        ], feed_dict={
            phonemes: np.ones((2, 200)),
            phonemes_seq_len: 8 * np.ones(2),
            speaker_ids: 2 * np.ones((2)),
            voiced_target: np.ones((2, 200)),
            frequency_target: np.random.rand(2, 200)
        })
        _, loss, global_step, summary  = out
        
        print global_step
        print loss


        train_writer.add_summary(summary, global_step)

        # detect gradient explosion
        if loss > 1e8 and global_step > 500:
            print('loss exploded')
            break

        if global_step % save_energy == 0 and global_step != 0:

            print('saving weights')
            if not os.path.exists('../weights/train_grapheme_to_phoneme_model_notebook/'):
                os.makedirs('../weights/train_grapheme_to_phoneme_model_notebook/')
            saver.save(sess, '../weights/train_grapheme_to_phoneme_model_notebook/', global_step=global_step)
    
    coord.request_stop()
    coord.join(threads)

1
2.20529
2
2.0784
saving weights
